In [1]:
import pickle
import time
import math
import random
import statistics

class HammingBatchAnalyzer:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_conflicts(self, S):
        count = 0
        for i in range(len(S)):
            for j in range(i + 1, len(S)):
                if not self.are_neighbors(S[i], S[j]):
                    count += 1
        return count

    def run_single_solve(self, run_id, target=16, max_iter=50000):
        # Parameters (Printing as requested)
        T = 5.0
        T_min = 0.0001
        alpha = 0.9997 
        penalty_factor = 2.0
        
        v_self = random.randint(1, self.num_nodes)
        neighbors = [i for i in range(1, self.num_nodes + 1) if i != v_self and self.are_neighbors(v_self, i)]
        random.shuffle(neighbors)
        S = [v_self] + neighbors[:10] 
        
        current_conflicts = self.get_conflicts(S)
        best_size = 0
        start_time = time.time()

        for i in range(max_iter):
            if T <= T_min: break
            
            new_S = list(S)
            rand_val = random.random()
            
            # Neighborhood Moves
            if rand_val < 0.3 and len(new_S) > 1:
                node_to_remove = random.choice([u for u in new_S if u != v_self])
                new_S.remove(node_to_remove)
            elif rand_val < 0.6:
                potential = [n for n in range(1, self.num_nodes + 1) if n not in new_S]
                if potential: new_S.append(random.choice(potential))
            else:
                if len(new_S) > 1:
                    node_to_remove = random.choice([u for u in new_S if u != v_self])
                    new_S.remove(node_to_remove)
                    potential = [n for n in range(1, self.num_nodes + 1) if n not in new_S]
                    if potential: new_S.append(random.choice(potential))

            new_conflicts = self.get_conflicts(new_S)
            delta_energy = (new_conflicts - current_conflicts) * penalty_factor - (len(new_S) - len(S))

            if delta_energy < 0 or random.random() < math.exp(-delta_energy / T):
                S = new_S
                current_conflicts = new_conflicts
            
            if current_conflicts == 0:
                if len(S) > best_size:
                    best_size = len(S)
                if best_size >= target:
                    break
            T *= alpha 

        end_time = time.time()
        return {
            "success": best_size >= target,
            "size": best_size,
            "time": end_time - start_time,
            "clique": tuple(sorted(S)) if current_conflicts == 0 else None
        }

    def batch_execute(self, num_runs=100, target=16):
        print("="*50)
        print(f"STARTING BATCH EXECUTION: {num_runs} Runs")
        print(f"Target Size: {target} | Alpha: 0.9997 | Initial T: 5.0")
        print("="*50)
        
        results = []
        for r in range(1, num_runs + 1):
            res = self.run_single_solve(r, target)
            results.append(res)
            if r % 10 == 0:
                print(f"Completed {r}/{num_runs} runs...")

        # Statistics Calculation
        times = [r['time'] for r in results]
        sizes = [r['size'] for r in results]
        successes = [r['success'] for r in results]
        unique_cliques = set([r['clique'] for r in results if r['clique'] is not None])
        
        # Diversification: All unique nodes found across all valid cliques
        all_nodes = set()
        for c in unique_cliques:
            all_nodes.update(c)

        print("\n" + "#"*50)
        print("FINAL STATISTICS & ANALYSIS")
        print("#"*50)
        print(f"Success Rate: {(sum(successes)/num_runs)*100:.2f}%")
        print(f"Avg Runtime: {statistics.mean(times):.4f}s (StdDev: {statistics.stdev(times):.4f}s)")
        print(f"Max Size Found: {max(sizes)}")
        print(f"Avg Size Found: {statistics.mean(sizes):.2f}")
        print("-" * 30)
        print(f"DIVERSIFICATION ANALYSIS")
        print(f"Unique Cliques Found: {len(unique_cliques)}")
        print(f"Total Unique Nodes Involved: {len(all_nodes)} / {self.num_nodes}")
        print(f"Exploration Coverage: {(len(all_nodes)/self.num_nodes)*100:.2f}%")
        print("#"*50)

# --- Run Analysis ---
analyzer = HammingBatchAnalyzer('hamming8-4_nam.data')
analyzer.batch_execute(num_runs=100, target=16)

STARTING BATCH EXECUTION: 100 Runs
Target Size: 16 | Alpha: 0.9997 | Initial T: 5.0
Completed 10/100 runs...
Completed 20/100 runs...
Completed 30/100 runs...
Completed 40/100 runs...
Completed 50/100 runs...
Completed 60/100 runs...
Completed 70/100 runs...
Completed 80/100 runs...
Completed 90/100 runs...
Completed 100/100 runs...

##################################################
FINAL STATISTICS & ANALYSIS
##################################################
Success Rate: 57.00%
Avg Runtime: 1.6083s (StdDev: 0.9143s)
Max Size Found: 16
Avg Size Found: 14.34
------------------------------
DIVERSIFICATION ANALYSIS
Unique Cliques Found: 97
Total Unique Nodes Involved: 256 / 256
Exploration Coverage: 100.00%
##################################################


In [3]:
import pickle
import time
import math
import random
import statistics

class TunedHammingSA:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u
        return ((self.codes_int[u] >> v) & 1) == 0

    def run_solve(self, target=16):
        # --- HYPER-PARAMETERS ---
        T = 2.5            # Slightly higher starting T for better initial mixing
        alpha = 0.9992     # Slightly faster cooling than before to avoid plateau idling
        max_iter = 40000
        pool_size = 8      # Increased pool size for better "Min-Conflict" selection
        
        nodes = list(range(1, self.num_nodes + 1))
        S = random.sample(nodes, 10) # Start with 10 random nodes
        
        best_size = 0
        start_time = time.time()

        for step in range(max_iter):
            if T < 0.00001: break
            
            # --- TUNED VIBRATION ---
            # High penalty for 85 steps, Low penalty for 15 steps
            penalty = 25.0 if (step % 100 < 85) else 3.5
            
            # 1. EVICITION: Identify node in S with most conflicts
            # (Local conflict check is faster than global)
            u = max(S, key=lambda z: sum(1 for m in S if z != m and not self.are_neighbors(z, m)))
            
            # 2. TUNED SELECTION: Min-Conflict from a larger pool
            pool = random.sample(nodes, pool_size)
            v = min(pool, key=lambda z: sum(1 for m in S if z != u and not self.are_neighbors(z, m)))
            
            # 3. CALCULATE DELTA
            curr_conflicts = sum(1 for i in range(len(S)) for j in range(i+1, len(S)) if not self.are_neighbors(S[i], S[j]))
            S_next = [n for n in S if n != u] + [v]
            next_conflicts = sum(1 for i in range(len(S_next)) for j in range(i+1, len(S_next)) if not self.are_neighbors(S_next[i], S_next[j]))
            
            # Energy Delta
            e_old = (curr_conflicts * penalty) - len(S)
            e_new = (next_conflicts * penalty) - len(S_next)
            delta_e = e_new - e_old

            # 4. METROPOLIS
            if delta_e <= 0 or random.random() < math.exp(-delta_e / T):
                S = S_next
                curr_conflicts = next_conflicts
            
            # 5. GROWTH & SUCCESS
            if curr_conflicts == 0:
                # Greedy Expansion
                for cand in nodes:
                    if cand not in S and all(self.are_neighbors(cand, m) for m in S):
                        S.append(cand)
                
                best_size = len(S)
                if best_size >= target:
                    return True, time.time() - start_time, tuple(sorted(S))

            T *= alpha 
        
        return False, time.time() - start_time, None

    def batch_test(self, num_runs=100):
        print(f"Tuning Test: 100 Runs on Hamming 8-4 (Target 16)")
        results = [self.run_solve() for _ in range(num_runs)]
        
        successes = [r for r in results if r[0]]
        times = [r[1] for r in successes]
        
        print("\n" + "="*40)
        print(f"SUCCESS RATE: {(len(successes)/num_runs)*100:.2f}%")
        if successes:
            print(f"AVG TIME: {statistics.mean(times):.4f}s")
            print(f"STDEV: {statistics.stdev(times):.4f}s")
            print(f"UNIQUE CLIQUES: {len(set(r[2] for r in successes))}")
        print("="*40)

# Execute
tester = TunedHammingSA('hamming8-4_nam.data')
tester.batch_test(100)

Tuning Test: 100 Runs on Hamming 8-4 (Target 16)

SUCCESS RATE: 95.00%
AVG TIME: 0.3863s
STDEV: 0.3590s
UNIQUE CLIQUES: 85


In [4]:
import pickle
import time
import math
import random
import statistics

class MaximumCliqueBatchTester:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def calculate_total_conflicts(self, S):
        conflicts = 0
        for i in range(len(S)):
            for j in range(i + 1, len(S)):
                if not self.are_neighbors(S[i], S[j]):
                    conflicts += 1
        return conflicts

    def solve_single(self, target=34, max_iter=200000):
        # Algorithm 3: Uniform start + No Penalty Oscillation
        seed_node = random.randint(1, self.num_nodes)
        S = [seed_node]
        all_nodes = list(range(1, self.num_nodes + 1))
        random.shuffle(all_nodes)
        for n in all_nodes:
            if n not in S and all(self.are_neighbors(n, member) for member in S):
                S.append(n)
        
        current_conflicts = 0
        T = 2.0
        alpha = 0.99995  # Slow cooling
        penalty_weight = 10.0 # Static Penalty
        
        start_t = time.time()
        for i in range(max_iter):
            if T < 0.001: break
            
            new_S = list(S)
            move_type = random.random()
            if move_type < 0.2 and len(new_S) > 1:
                new_S.remove(random.choice(new_S))
            elif move_type < 0.7:
                candidates = random.sample(range(1, self.num_nodes + 1), 15)
                best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                if best_cand not in new_S: new_S.append(best_cand)
            else:
                if len(new_S) > 1:
                    new_S.remove(random.choice(new_S))
                    candidates = random.sample(range(1, self.num_nodes + 1), 15)
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    if best_cand not in new_S: new_S.append(best_cand)

            new_conflicts = self.calculate_total_conflicts(new_S)
            delta = (new_conflicts * penalty_weight - len(new_S)) - (current_conflicts * penalty_weight - len(S))

            if delta < 0 or random.random() < math.exp(-delta / T):
                S = new_S
                current_conflicts = new_conflicts
            
            if current_conflicts == 0 and len(S) >= target:
                return True, len(S), time.time() - start_t, tuple(sorted(S))
            
            T *= alpha
        return False, len(S), time.time() - start_t, tuple(sorted(S)) if current_conflicts == 0 else None

    def run_batch(self, num_runs=100):
        print(f"Batch Running C125-9 (Target 34) | Baseline Algorithm 3")
        results = []
        for i in range(num_runs):
            results.append(self.solve_single())
            if (i+1) % 10 == 0: print(f"Completed {i+1}/100...")

        successes = [r for r in results if r[0]]
        times = [r[2] for r in successes]
        unique_cliques = set([r[3] for r in results if r[3] is not None and r[1] >= 30])
        all_nodes_found = set().union(*[set(c) for c in unique_cliques])

        print("\n" + "#"*40)
        print("BASELINE RESULTS: C125-9")
        print("#"*40)
        print(f"Success Rate: {len(successes)}%")
        print(f"Avg Time (Success): {statistics.mean(times) if times else 0:.4f}s")
        print(f"Unique Cliques Found (Size >= 30): {len(unique_cliques)}")
        print(f"Exploration Coverage: {(len(all_nodes_found)/self.num_nodes)*100:.2f}%")
        print("#"*40)

if __name__ == "__main__":
    tester = MaximumCliqueBatchTester('C125-9_nam.data')
    tester.run_batch(100)

Batch Running C125-9 (Target 34) | Baseline Algorithm 3
Completed 10/100...
Completed 20/100...
Completed 30/100...
Completed 40/100...
Completed 50/100...
Completed 60/100...
Completed 70/100...
Completed 80/100...
Completed 90/100...
Completed 100/100...

########################################
BASELINE RESULTS: C125-9
########################################
Success Rate: 100%
Avg Time (Success): 1.5997s
Unique Cliques Found (Size >= 30): 97
Exploration Coverage: 49.60%
########################################


In [5]:
import pickle
import time
import math
import random
import statistics
import networkx as nx

class Algorithm4BatchTester:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        self.weights = self._precompute_weights()

    def are_neighbors(self, u, v):
        if u == v: return False
        return ((self.codes_int[u] >> v) & 1) == 0

    def _precompute_weights(self):
        G = nx.Graph()
        for u in range(1, self.num_nodes + 1):
            for v in range(u + 1, self.num_nodes + 1):
                if self.are_neighbors(u, v): G.add_edge(u, v)
        pr = nx.pagerank(G, alpha=0.85)
        return [pr.get(i, 0)**3 for i in range(self.num_nodes + 1)]

    def run_single(self, target=34, max_iter=100000):
        nodes = list(range(1, self.num_nodes + 1))
        w_sum = sum(self.weights[1:])
        norm_weights = [w / w_sum for w in self.weights[1:]]
        
        # PR-Biased Initial State
        S = random.choices(nodes, weights=norm_weights, k=15)
        T, alpha = 2.0, 0.9999
        start_t = time.time()

        for step in range(max_iter):
            if T < 0.001: break
            penalty = 25.0 if (step % 100 < 85) else 3.5 # Oscillation
            
            # Min-Conflict / PR-Pool Logic
            u = max(S, key=lambda z: sum(1 for m in S if z != m and not self.are_neighbors(z, m)))
            pool = random.choices(nodes, weights=norm_weights, k=10)
            v = min(pool, key=lambda z: sum(1 for m in S if z != u and not self.are_neighbors(z, m)))
            
            # Energy Calculation
            curr_c = sum(1 for i in range(len(S)) for j in range(i+1, len(S)) if not self.are_neighbors(S[i], S[j]))
            S_next = [n for n in S if n != u] + [v]
            next_c = sum(1 for i in range(len(S_next)) for j in range(i+1, len(S_next)) if not self.are_neighbors(S_next[i], S_next[j]))
            
            delta = (next_c * penalty - len(S_next)) - (curr_c * penalty - len(S))
            if delta <= 0 or random.random() < math.exp(-delta / T):
                S, curr_c = S_next, next_c
            
            if curr_c == 0:
                for cand in nodes:
                    if cand not in S and all(self.are_neighbors(cand, m) for m in S): S.append(cand)
                if len(S) >= target: return True, len(S), time.time() - start_t, tuple(sorted(S))
            T *= alpha
        return False, len(S), time.time() - start_t, None

    def execute_batch(self, num_runs=100):
        print(f"Batch Running C125-9 | Algorithm 4 (PageRank + Oscillation)")
        results = [self.run_single() for _ in range(num_runs)]
        successes = [r for r in results if r[0]]
        times = [r[2] for r in successes]
        unique_cliques = set([r[3] for r in results if r[3] is not None])
        
        print("\n" + "="*40)
        print("ALGORITHM 4 FINAL BATCH RESULTS")
        print("="*40)
        print(f"Success Rate: {len(successes)}%")
        print(f"Avg Success Time: {statistics.mean(times):.4f}s")
        print(f"Unique Max Cliques Found: {len(unique_cliques)}")
        print(f"Structural Coverage: {(len(set().union(*[set(c) for c in unique_cliques]))/self.num_nodes)*100:.2f}%")
        print("="*40)

tester = Algorithm4BatchTester('C125-9_nam.data')
tester.execute_batch(100)

Batch Running C125-9 | Algorithm 4 (PageRank + Oscillation)

ALGORITHM 4 FINAL BATCH RESULTS
Success Rate: 96%
Avg Success Time: 8.5415s
Unique Max Cliques Found: 91
Structural Coverage: 50.40%


In [6]:
import pickle
import time
import math
import random
from collections import deque
import networkx as nx
import statistics

class BrockBatchTester:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        
        # Spectral Analysis
        G = nx.Graph()
        for u in range(1, self.num_nodes + 1):
            for v in range(u + 1, self.num_nodes + 1):
                if ((self.codes_int[u] >> v) & 1) == 0:
                    G.add_edge(u, v)
        
        pr_scores = nx.pagerank(G, alpha=0.85)
        self.nodes_list = list(range(1, self.num_nodes + 1))
        # PR^3 for high-density focus
        self.weights = [pr_scores.get(i, 0)**3 for i in self.nodes_list]

    def are_neighbors(self, u, v):
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def solve_once(self, target=12, max_steps=40000):
        S = random.choices(self.nodes_list, weights=self.weights, k=1)
        tabu = deque(maxlen=55)
        T, alpha = 2.0, 0.99995
        current_conflicts = 0
        start_time = time.time()
        
        for i in range(max_steps):
            # Penalty Oscillation (Thermodynamic Vibration)
            penalty = 22.0 if (i // 1000) % 2 == 0 else 4.0
            
            # --- CLOSE-OUT SCAN (Deterministic completion) ---
            if len(S) == target - 1 and current_conflicts == 0:
                for candidate in range(1, self.num_nodes + 1):
                    if candidate not in S and self.get_node_conflicts(candidate, S) == 0:
                        S.append(candidate)
                        return True, len(S), time.time() - start_time, tuple(sorted(S))

            new_S = list(S)
            move = random.random()

            if move < 0.12 and len(new_S) > 1: # Remove
                node = random.choice(new_S)
                new_S.remove(node)
                tabu.append(node)
            elif move < 0.55: # PR-Biased Add
                potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                candidates = [n for n in potential if n not in tabu and n not in new_S]
                if candidates:
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    new_S.append(best_cand)
            else: # PR-Biased Swap
                if len(new_S) > 1:
                    rem = random.choice(new_S)
                    new_S.remove(rem)
                    tabu.append(rem)
                    potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                    candidates = [n for n in potential if n not in tabu and n not in new_S]
                    if candidates:
                        add = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                        new_S.append(add)

            # Energy Logic
            new_conf = sum(self.get_node_conflicts(new_S[j], new_S[j+1:]) for j in range(len(new_S)))
            delta = (new_conf * penalty - len(new_S)) - (current_conflicts * penalty - len(S))

            if delta < 0 or random.random() < math.exp(-delta / T):
                S = new_S
                current_conflicts = new_conf
            
            T *= alpha
            if T < 0.02: break

        return False, len(S), time.time() - start_time, None

    def batch_run(self, num_runs=100, target=12):
        print(f"BATCH RUN: Brock200_2 (Target {target}) | 100 Runs")
        results = []
        for i in range(num_runs):
            res = self.solve_once(target)
            results.append(res)
            if (i+1) % 10 == 0: print(f"Progress: {i+1}/100")

        successes = [r for r in results if r[0]]
        times = [r[2] for r in successes]
        unique_cliques = set([r[3] for r in results if r[3] is not None])
        
        print("\n" + "#"*40)
        print("ALGORITHM 4.1 BATCH RESULTS")
        print("#"*40)
        print(f"Success Rate: {len(successes)}%")
        if successes:
            print(f"Avg Solve Time: {statistics.mean(times):.4f}s")
            print(f"Unique Max Cliques Found: {len(unique_cliques)}")
            all_nodes = set().union(*[set(c) for c in unique_cliques])
            print(f"Spectral Coverage: {(len(all_nodes)/self.num_nodes)*100:.2f}%")
        print("#"*40)

if __name__ == "__main__":
    tester = BrockBatchTester('brock200_2_nam.data')
    tester.batch_run(100)

BATCH RUN: Brock200_2 (Target 12) | 100 Runs
Progress: 10/100
Progress: 20/100
Progress: 30/100
Progress: 40/100
Progress: 50/100
Progress: 60/100
Progress: 70/100
Progress: 80/100
Progress: 90/100
Progress: 100/100

########################################
ALGORITHM 4.1 BATCH RESULTS
########################################
Success Rate: 0%
########################################


In [1]:
import pickle
import time
import math
import random
from collections import deque
import networkx as nx
import statistics

class BrockBatchRefined:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        
        # Spectral Analysis (Keep the PageRank bias)
        G = nx.Graph()
        for u in range(1, self.num_nodes + 1):
            for v in range(u + 1, self.num_nodes + 1):
                if ((self.codes_int[u] >> v) & 1) == 0:
                    G.add_edge(u, v)
        
        pr_scores = nx.pagerank(G, alpha=0.85)
        self.nodes_list = list(range(1, self.num_nodes + 1))
        self.weights = [pr_scores.get(i, 0)**3 for i in self.nodes_list]

    def are_neighbors(self, u, v):
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def solve_attempt(self, target, max_steps):
        """A single SA shot. Returns (Success, BestSize, Clique)"""
        S = random.choices(self.nodes_list, weights=self.weights, k=1)
        tabu = deque(maxlen=55)
        T, alpha = 2.0, 0.99995
        current_conflicts = 0
        local_best = 1
        
        for i in range(max_steps):
            penalty = 22.0 if (i // 1000) % 2 == 0 else 4.0
            
            # Close-out scan
            if len(S) == target - 1 and current_conflicts == 0:
                for candidate in self.nodes_list:
                    if candidate not in S and self.get_node_conflicts(candidate, S) == 0:
                        S.append(candidate)
                        return True, len(S), tuple(sorted(S))

            new_S = list(S)
            move = random.random()
            if move < 0.12 and len(new_S) > 1: # Remove
                node = random.choice(new_S)
                new_S.remove(node)
                tabu.append(node)
            elif move < 0.55: # Add
                potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                candidates = [n for n in potential if n not in tabu and n not in new_S]
                if candidates:
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    new_S.append(best_cand)
            else: # Swap
                if len(new_S) > 1:
                    rem = random.choice(new_S)
                    new_S.remove(rem)
                    tabu.append(rem)
                    potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                    candidates = [n for n in potential if n not in tabu and n not in new_S]
                    if candidates:
                        add = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                        new_S.append(add)

            new_conf = sum(self.get_node_conflicts(new_S[j], new_S[j+1:]) for j in range(len(new_S)))
            delta = (new_conf * penalty - len(new_S)) - (current_conflicts * penalty - len(S))
            if delta < 0 or random.random() < math.exp(-delta / T):
                S, current_conflicts = new_S, new_conf
            
            if current_conflicts == 0: local_best = max(local_best, len(S))
            T *= alpha
            if T < 0.02: break
        return False, local_best, None

    def execute_global_run(self, target):
        """Runs attempts until success. This matches your 'Original Run'."""
        start = time.time()
        attempts = 0
        while True:
            attempts += 1
            success, best, clique = self.solve_attempt(target, 40000)
            if success:
                return time.time() - start, attempts, clique

    def run_batch(self, num_samples=10, target=12):
        print(f"--- BATCH EVALUATION: {num_samples} FULL SUCCESSES ---")
        times, attempt_counts = [], []
        
        for i in range(num_samples):
            duration, atts, _ = self.execute_global_run(target)
            times.append(duration)
            attempt_counts.append(atts)
            print(f"Sample {i+1}: Found in {duration:.2f}s ({atts} attempts)")
            
        print("\n" + "="*40)
        print("BROCK PERFORMANCE SUMMARY")
        print("="*40)
        print(f"Avg Time to Success: {statistics.mean(times):.2f}s")
        print(f"Avg Attempts Required: {statistics.mean(attempt_counts):.1f}")
        print(f"Reliability (Success per attempt): {(1/statistics.mean(attempt_counts))*100:.2f}%")
        print("="*40)

if __name__ == "__main__":
    tester = BrockBatchRefined('brock200_2_nam.data')
    # Use 10 samples instead of 100 because Brock takes ~120s per success
    tester.run_batch(num_samples=10, target=12)

--- BATCH EVALUATION: 10 FULL SUCCESSES ---


KeyboardInterrupt: 

In [2]:
import pickle
import time
import math
import random
from collections import deque
import networkx as nx
import statistics

class BrockOptimizedBatch:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        
        # 1. PageRank with Soft Bias (Optimized for Brock)
        G = nx.Graph()
        for u in range(1, self.num_nodes + 1):
            for v in range(u + 1, self.num_nodes + 1):
                if ((self.codes_int[u] >> v) & 1) == 0:
                    G.add_edge(u, v)
        
        pr_scores = nx.pagerank(G, alpha=0.85)
        self.nodes_list = list(range(1, self.num_nodes + 1))
        # Use 1.2 instead of 3.0 to include 'spectral outliers' in the clique
        self.weights = [pr_scores.get(i, 0)**1.2 for i in self.nodes_list]

    def are_neighbors(self, u, v):
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def solve_attempt(self, target, max_steps):
        # Start with a biased random node
        S = random.choices(self.nodes_list, weights=self.weights, k=1)
        tabu = deque(maxlen=80) # Increased to force exploration
        T, alpha = 2.5, 0.99992 # Refined cooling schedule
        current_conflicts = 0
        
        for i in range(max_steps):
            # Strategic Oscillation
            penalty = 25.0 if (i // 1000) % 2 == 0 else 3.5
            
            # Close-out scan (One-node-to-victory check)
            if len(S) >= target - 1 and current_conflicts == 0:
                for candidate in self.nodes_list:
                    if candidate not in S and self.get_node_conflicts(candidate, S) == 0:
                        S.append(candidate)
                        if len(S) >= target: return True, S

            new_S = list(S)
            move = random.random()

            if move < 0.15 and len(new_S) > 0: # Removal
                node = random.choice(new_S)
                new_S.remove(node)
                tabu.append(node)
            elif move < 0.60: # Expansion
                potential = random.choices(self.nodes_list, weights=self.weights, k=60)
                candidates = [n for n in potential if n not in tabu and n not in new_S]
                if candidates:
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    new_S.append(best_cand)
            else: # Swap
                if len(new_S) > 0:
                    rem = random.choice(new_S)
                    new_S.remove(rem)
                    tabu.append(rem)
                    potential = random.choices(self.nodes_list, weights=self.weights, k=60)
                    candidates = [n for n in potential if n not in tabu and n not in new_S]
                    if candidates:
                        add = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                        new_S.append(add)

            new_conf = sum(self.get_node_conflicts(new_S[j], new_S[j+1:]) for j in range(len(new_S)))
            delta = (new_conf * penalty - len(new_S)) - (current_conflicts * penalty - len(S))
            
            if delta < 0 or (T > 0 and random.random() < math.exp(-delta / T)):
                S, current_conflicts = new_S, new_conf
            
            T *= alpha
            if T < 0.01: break
        return False, None

    def execute_global_run(self, target):
        start = time.time()
        attempts = 0
        while True:
            attempts += 1
            success, clique = self.solve_attempt(target, 35000) # Slightly faster per attempt
            if success:
                return time.time() - start, attempts, clique

    def run_batch(self, num_samples=5, target=12):
        print(f"--- ENHANCED BATCH: {num_samples} SUCCESSES (SOFT BIAS) ---")
        times, attempt_counts = [], []
        
        for i in range(num_samples):
            duration, atts, _ = self.execute_global_run(target)
            times.append(duration)
            attempt_counts.append(atts)
            print(f"Sample {i+1}: Found in {duration:.2f}s ({atts} attempts)")
            
        print("\n" + "="*40)
        print("BROCK OPTIMIZED PERFORMANCE SUMMARY")
        print("="*40)
        print(f"Avg Time to Success: {statistics.mean(times):.2f}s")
        print(f"Avg Attempts Required: {statistics.mean(attempt_counts):.1f}")
        print(f"Reliability (Prob per attempt): {(1/statistics.mean(attempt_counts))*100:.2f}%")
        print("="*40)

if __name__ == "__main__":
    tester = BrockOptimizedBatch('brock200_2_nam.data')
    tester.run_batch(num_samples=5, target=12)

--- ENHANCED BATCH: 5 SUCCESSES (SOFT BIAS) ---
Sample 1: Found in 152.49s (12 attempts)
Sample 2: Found in 505.28s (39 attempts)
Sample 3: Found in 888.44s (74 attempts)
Sample 4: Found in 281.31s (24 attempts)
Sample 5: Found in 160.84s (14 attempts)

BROCK OPTIMIZED PERFORMANCE SUMMARY
Avg Time to Success: 397.67s
Avg Attempts Required: 32.6
Reliability (Prob per attempt): 3.07%
